# Debug training
May 19, 2021

In [1]:
import     numpy as np
import matplotlib.pyplot as plt
import pandas as pd

import subprocess as sp
import sys
import os
import glob
import pickle 

from matplotlib.colors import LogNorm, PowerNorm, Normalize
import seaborn as sns
from functools import reduce

import socket

In [2]:
from ipywidgets import *

In [3]:
%matplotlib widget

In [4]:
base_dict={'cori':'/global/u1/v/vpa/project/jpt_notebooks/Cosmology/Cosmo_GAN/repositories/cosmogan_pytorch/',
         'summit':'/autofs/nccs-svm1_home1/venkitesh/projects/cosmogan/cosmogan_pytorch/'}
facility='cori' if socket.gethostname()[:4]=='cori' else 'summit'

base_dir=base_dict[facility]

In [5]:
sys.path.append(base_dir+'/code/modules_image_analysis/')
from modules_img_analysis import *
# sys.path.append(base_dir+'/code/5_3d_cgan/1_main_code/')
# import post_analysis_pandas as post

In [6]:
### Transformation functions for image pixel values
def f_transform(x):
    return 2.*x/(x + 4.) - 1.

def f_invtransform(s):
    return 4.*(1. + s)/(1. - s)


## Read data

In [7]:
dict1={'cori':{
'2d':'/global/cfs/cdirs/m3363/vayyar/cosmogan_data/results_from_other_code/pytorch/results/128_square/',
'3d':'/global/cfs/cdirs/m3363/vayyar/cosmogan_data/results_from_other_code/pytorch/results/3D/',
'3d_cgan':'/global/cfs/cdirs/m3363/vayyar/cosmogan_data/results_from_other_code/pytorch/results/3d_cGAN/'},
'summit':{'2d':'/gpfs/alpine/ast153/proj-shared/venkitesh/Cosmogan/data/results_pytorch/2d/',
          '3d':'/gpfs/alpine/ast153/proj-shared/venkitesh/Cosmogan/data/results_pytorch/3d/'}}

In [13]:
# parent_dir=u.result
parent_dir=dict1[facility]['3d']
dir_lst=[i.split('/')[-1] for i in glob.glob(parent_dir+'20210521*')]
dir_lst
w=interactive(lambda x: x, x=Dropdown(options=dir_lst))
display(w)

interactive(children=(Dropdown(description='x', options=('20210521_94739_cgan_bs16_nodes8_lrg0.0015_lrdfastvar…

In [14]:
result=w.result
result_dir=parent_dir+result
print(result_dir)

/gpfs/alpine/ast153/proj-shared/venkitesh/Cosmogan/data/results_pytorch/3d/20210521_134301_cgan_bs16_nodes8_lrg0.003_lrdfastvary


## Plot Losses

In [15]:
df_metrics=pd.read_pickle(result_dir+'/df_metrics.pkle').astype(np.float64)
# df_metrics.tail(10)
def f_plot_metrics(df,col_list):
    
    plt.figure()
    for key in col_list:
        plt.plot(df_metrics[key],label=key,marker='*',linestyle='')
    plt.legend()
    
#     col_list=list(col_list)
#     df.plot(kind='line',x='step',y=col_list)

f_plot_metrics(df_metrics,['hist_chi'])

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [16]:
f_plot_metrics(df_metrics,['lr_d','lr_g'])

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [23]:
interact_manual(f_plot_metrics,df=fixed(df_metrics), col_list=SelectMultiple(options=df_metrics.columns.values))

interactive(children=(SelectMultiple(description='col_list', options=('step', 'epoch', 'Dreal', 'Dfake', 'Dful…

<function __main__.f_plot_metrics(df, col_list)>

In [12]:
# df_metrics[(df_metrics.lr_d>=6.69e-04) ]

In [37]:
df_metrics.plot(kind='line',x='step',y=['lr_d'])

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

<AxesSubplot:xlabel='step'>

In [15]:
np.unique(df_metrics.lr_d.values),np.unique(df_metrics.lr_g.values)

# display(df_metrics.sort_values(by=['hist_chi']).head(8))
# display(df_metrics.sort_values(by=['spec_chi']).head(8))

(array([0.00020046, 0.00024506, 0.00029958, 0.00036624, 0.00044773,
        0.00054734, 0.00066912, 0.000818  , 0.001     ,        nan]),
 array([0.00020046, 0.00024506, 0.00029958, 0.00036624, 0.00044773,
        0.00054734, 0.00066912, 0.000818  , 0.001     ,        nan]))

## Calculating learn rates

In [21]:
Nsteps=6;Lf=0.0003;Li=0.001
Lambda=(Lf/Li)**(1.0/Nsteps)
print(Lambda,np.sqrt(Lambda))

lst=[5,10,15,25,35,100]
# lst=range(1,11)
[(Li*Lambda**(count+1),i) for count,i in enumerate(lst)]

0.818188823000027 0.9045379057839572


[(0.000818188823000027, 5),
 (0.0006694329500821696, 10),
 (0.0005477225575051662, 15),
 (0.00044814047465571656, 25),
 (0.0003666635274972342, 35),
 (0.0003000000000000001, 100)]

## Grid plot of images

In [46]:
epoch=170
flist=glob.glob(result_dir+'/images/gen_img*_epoch-{0}_step*'.format(epoch))
steps_list=[fname.split('/')[-1].split('step-')[-1].split('.')[0] for fname in flist]

print(*steps_list)

9530 9540 9540 9540 9550 9530 9570 9570 9530 9560 9560 9550 9570 9550 9560


In [47]:
# fname=flist[0]
# fname,fname.split('/')[-1].split('step-')[-1].split('.')[0]

step=9550
fname=glob.glob(result_dir+'/images/gen_img_*_epoch-{0}_step-{1}.npy'.format(epoch,step))[0]
fname

'/gpfs/alpine/ast153/proj-shared/venkitesh/Cosmogan/data/results_pytorch/3d/20210519_81818_cgan_bs16_lr0.001_nodes8_spec0.1/images/gen_img_label-0.5_epoch-170_step-9550.npy'

In [48]:
images=np.load(fname)[:,0,:,:]
print(images.shape)
f_plot_grid(images[:8,:,:,0],cols=4,fig_size=(8,4))

(32, 64, 64, 64)
2 4


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …